Begin by importing the libraries needed for data cleaning and preparation.

In [1]:
import numpy as np
import pandas as pd


#### Data was obtained from the MIMIC III database (https://mimic.physionet.org/).

The first step is to read in the first dataset of interest, ICUSTAYS, and have a first look at the columns and rows. <br> 


Remove patients who did not spend time in the ICU, since focus of this project is on ICU patients.

In [2]:
#read in ICUSTAYS dataset
icu_stays = pd.read_csv('ICUSTAYS.csv.gz', compression='gzip' )
print(icu_stays.shape)

(61532, 12)


#### Columns of interest in the ICU dataset: 
- INTIME
- HADM_ID

In [3]:
#convert INTIME to datetime format. The errors='coerce' argument allows for missing values
icu_stays.INTIME = pd.to_datetime(icu_stays.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing intime:', icu_stays.INTIME.isnull().sum())


Number of missing intime: 0


In [4]:
#reduce the columns to only 'HADM_ID' and 'INTIME'
icu_stays = icu_stays[['HADM_ID', 'INTIME']]
#icu_stays.head()

In [5]:
#add column ICU to indicate whether or not patient had ICU stay (all values will be true)
icu_stays['ICU'] = True
icu_stays = icu_stays[['HADM_ID', 'ICU']]
#icu_stays.head()


In [6]:
#drop duplicate rows
print('There are ', len(icu_stays[icu_stays.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are  3746 duplicate hospital admissions.


These admissions are patients who stayed more than once in the ICU on that admission.

In [7]:
icu_stays = icu_stays.drop_duplicates()

In [8]:
print('There are now', len(icu_stays[icu_stays.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are now 0 duplicate hospital admissions.


In [9]:
icu_stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 61531
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   HADM_ID  57786 non-null  int64
 1   ICU      57786 non-null  bool 
dtypes: bool(1), int64(1)
memory usage: 959.3 KB


In [10]:
admissions = pd.read_csv('ADMISSIONS.csv.gz', compression='gzip')
print(admissions.shape)


(58976, 19)


There are 58976 rows and 19 columns.

#### Columns of interest (or may be of interest) from the 'admissions' dataset include:
- HOSPITAL_EXPIRE_FLAG
- SUBJECT_ID
- HADM_ID
- ADMITTIME
- DEATHTIME
- ADMISSION_TYPE

In [11]:
# exploring the data to determine the datatypes, in particular, of the date columns.
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                58976 non-null  int64 
 1   SUBJECT_ID            58976 non-null  int64 
 2   HADM_ID               58976 non-null  int64 
 3   ADMITTIME             58976 non-null  object
 4   DISCHTIME             58976 non-null  object
 5   DEATHTIME             5854 non-null   object
 6   ADMISSION_TYPE        58976 non-null  object
 7   ADMISSION_LOCATION    58976 non-null  object
 8   DISCHARGE_LOCATION    58976 non-null  object
 9   INSURANCE             58976 non-null  object
 10  LANGUAGE              33644 non-null  object
 11  RELIGION              58518 non-null  object
 12  MARITAL_STATUS        48848 non-null  object
 13  ETHNICITY             58976 non-null  object
 14  EDREGTIME             30877 non-null  object
 15  EDOUTTIME             30877 non-null

Next, convert the dates to datetime format for processing.

In [12]:
#convert ADMITTIME and DEATHTIME to datetime format. The errors='coerce' argument allows for missing values
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing date admissions:', admissions.ADMITTIME.isnull().sum())
print('Number of missing DEATHTIME:', admissions.DEATHTIME.isnull().sum())
print('Number of missing DISCHTIME:', admissions.DISCHTIME.isnull().sum())


Number of missing date admissions: 0
Number of missing DEATHTIME: 53122
Number of missing DISCHTIME: 0


Now update the admissions dataset to include only HADM_ID that are in the icu_stays dataframe. Do this by completing a left merge on HADM_ID.

In [13]:
admissions.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')

In [14]:
admissions = pd.merge(icu_stays[['HADM_ID']], admissions[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']],\
            on=('HADM_ID') , how='left')

In [15]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 57785
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   HADM_ID               57786 non-null  int64         
 1   ROW_ID                57786 non-null  int64         
 2   SUBJECT_ID            57786 non-null  int64         
 3   ADMITTIME             57786 non-null  datetime64[ns]
 4   DISCHTIME             57786 non-null  datetime64[ns]
 5   DEATHTIME             5813 non-null   datetime64[ns]
 6   ADMISSION_TYPE        57786 non-null  object        
 7   ADMISSION_LOCATION    57786 non-null  object        
 8   DISCHARGE_LOCATION    57786 non-null  object        
 9   INSURANCE             57786 non-null  object        
 10  LANGUAGE              32990 non-null  object        
 11  RELIGION              57330 non-null  object        
 12  MARITAL_STATUS        47796 non-null  object        
 13  ETHNICITY       

Next a new column ENDTIME will be created. This will indicate the when to end note collection for analysis - i.e., 24 hours after ADMITTIME or at time of death - whichever is sooner.


In [16]:
admissions['ADMIT+24'] = admissions['ADMITTIME'] + pd.DateOffset(days=1)
admissions['ENDTIME'] = admissions['ADMIT+24']

admissions.ENDTIME = np.where(admissions['DEATHTIME'] < admissions['ENDTIME'], admissions['DEATHTIME']+pd.DateOffset(hours=-2), admissions['ENDTIME'])
#admissions.head()

In [17]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 57785
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   HADM_ID               57786 non-null  int64         
 1   ROW_ID                57786 non-null  int64         
 2   SUBJECT_ID            57786 non-null  int64         
 3   ADMITTIME             57786 non-null  datetime64[ns]
 4   DISCHTIME             57786 non-null  datetime64[ns]
 5   DEATHTIME             5813 non-null   datetime64[ns]
 6   ADMISSION_TYPE        57786 non-null  object        
 7   ADMISSION_LOCATION    57786 non-null  object        
 8   DISCHARGE_LOCATION    57786 non-null  object        
 9   INSURANCE             57786 non-null  object        
 10  LANGUAGE              32990 non-null  object        
 11  RELIGION              57330 non-null  object        
 12  MARITAL_STATUS        47796 non-null  object        
 13  ETHNICITY       

Twenty-four hours of notes need to be available in order to compare samples equally. Therefore, patients with a length of stay 24 hours or less are removed.

In [18]:
# first create a column that calculated length of stay
admissions['LOS_HOSP'] = admissions['DISCHTIME'] - admissions['ADMITTIME']
#admissions.head(1)

In [19]:
print('Number of hospitalizations more than 24 hours: ', len(admissions.loc[admissions['LOS_HOSP']>'24 hours']))
admissions = admissions.loc[admissions['LOS_HOSP']>'1 day']
admissions.shape


Number of hospitalizations more than 24 hours:  55727


(55727, 22)

In [20]:
# The next step is to add the target variable 'DEATH' which indicates whether or not a hospitalization resulted in death.
# positive death=1/negative death=0

#admissions['DEATH'] = admissions['DEATHTIME']>pd.Timestamp('00:00:00')
#admissions.DEATH.value_counts()
#print(admissions.DEATH.value_counts())
#print(admissions.DEATH.shape)

In [21]:
# I later realized that the column "HOSPITAL_EXPIRE_FLAG" is same as the "DEATH" column that was created.
# drop DEATH column
#admissions.drop('DEATH', axis=1, inplace=True)
#admissions.columns


In [22]:
admissions.shape

(55727, 22)

In [23]:
admissions.HOSPITAL_EXPIRE_FLAG.value_counts()

0    50856
1     4871
Name: HOSPITAL_EXPIRE_FLAG, dtype: int64

4871 out of 55727 hospital admissions resulted in death (8.7%). As should be expected, this dataset is imbalanced.

Now it's time to examine the caregiver notes. <br>
<br>
### Next: Read in the caregiver notes dataset and get an overview of its rows and features.


In [24]:
#Read in the caregiver notes dataset.

notes = pd.read_csv('NOTEEVENTS.csv.gz', compression='gzip')
#notes.head(2)

C:\Users\caras\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


ISERROR
A ‘1’ in the ISERROR column indicates that a physician has identified this note as an error. These rows will be removed.

In [26]:
notes.ISERROR.value_counts()

1.0    886
Name: ISERROR, dtype: int64

In [27]:
notes = notes.loc[notes['ISERROR']!=1]
notes.shape

(2082294, 11)

Columns of interest from the NOTES dataset:
- SUBJECT_ID
- HADM_ID
- CHARTDATE and CHARTTIME
- CATEGORY
- TEXT

In [28]:
# Viewing categories of caregiver notes to better understand the dataset.
notes.CATEGORY.value_counts()


Nursing/other        822497
Radiology            522279
Nursing              223182
ECG                  209051
Physician            141281
Discharge summary     59652
Echo                  45794
Respiratory           31701
Nutrition              9400
General                8236
Rehab Services         5408
Social Work            2661
Case Management         953
Pharmacy                101
Consult                  98
Name: CATEGORY, dtype: int64

In [29]:
notes.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082294 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 190.6+ MB


CHARTDATE records the date at which the note was charted. CHARTDATE will always have a time value of 00:00:00.

CHARTTIME records the date and time at which the note was charted. If both CHARTDATE and CHARTTIME exist, then the date portions will be identical. All records have a CHARTDATE. A subset are missing CHARTTIME. More specifically, notes with a CATEGORY value of ‘Discharge Summary’, ‘ECG’, and ‘Echo’ never have a CHARTTIME, only CHARTDATE. Other categories almost always have both CHARTTIME and CHARTDATE, but there is a small amount of missing data for CHARTTIME (usually less than 0.5% of the total number of notes for that category).

In [30]:
#convert the chart dates and times into datetime format

notes.CHARTTIME = pd.to_datetime(notes.CHARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
notes.CHARTDATE = pd.to_datetime(notes.CHARTDATE, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')


# check to see if there are any null dates
print('Number of missing CHARTTIME:', notes.CHARTTIME.isnull().sum())
print('Number of missing CHARTDATE:', notes.CHARTDATE.isnull().sum())



Number of missing CHARTTIME: 316566
Number of missing CHARTDATE: 0


Since there are so many missing CHARTTIME values, CHARTDATE will be used for processing and analysis (below). Before analyzing further, the dataframes will be merged.

In [31]:
admissions.columns

Index(['HADM_ID', 'ROW_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ADMIT+24', 'ENDTIME',
       'LOS_HOSP'],
      dtype='object')

In [32]:
# Merging the 'admissions' and 'notes' together. A left merge is used so that all rows for hospital admissions are included
# and any caregiver notes that are not associated with a hospital admission are dropped.
df=pd.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'LOS_HOSP','ADMITTIME', 'HOSPITAL_EXPIRE_FLAG', 'ADMISSION_TYPE', 'DEATHTIME','ADMIT+24', 'ENDTIME']],\
            notes[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY','TEXT']], \
            on=('HADM_ID', 'SUBJECT_ID') , how='left', suffixes=('adm','note'))
print('There are ', len(df), 'rows and ',len(df.columns), 'columns.')


There are  1830890 rows and  12 columns.


In [33]:
list(df.columns)

['SUBJECT_ID',
 'HADM_ID',
 'LOS_HOSP',
 'ADMITTIME',
 'HOSPITAL_EXPIRE_FLAG',
 'ADMISSION_TYPE',
 'DEATHTIME',
 'ADMIT+24',
 'ENDTIME',
 'CHARTDATE',
 'CATEGORY',
 'TEXT']

Next update the dataframe to only include notes taken within 24 hours of admission, or up until 2 hours before time of death if patient expired in the first 24 hours.

In [34]:

df = df[df['CHARTDATE'] <= df['ENDTIME']]

In [35]:
df.shape

(428039, 12)

In order to combine all the text samples into one single TEXT row per admission, a new dataframe, 'text' is created. Next it will be merged back with the original dataframe with  CHARTTIME and CATEGORY columns dropped.

In [36]:
text = df[['HADM_ID', 'TEXT']]

In [37]:
grouped_HADM = text.groupby("HADM_ID")

grouped_text = grouped_HADM["TEXT"].agg(lambda column: "".join(column))

grouped_text = grouped_text.reset_index(name="TEXT")

#print(grouped_text.head())


In [38]:
grouped_text.shape

(53977, 2)

In [39]:
df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'LOS_HOSP', 'ADMITTIME',
       'HOSPITAL_EXPIRE_FLAG', 'ADMISSION_TYPE', 'DEATHTIME', 'ADMIT+24',
       'ENDTIME', 'CHARTDATE', 'CATEGORY', 'TEXT'],
      dtype='object')

In [40]:
# Merging the 'grouped_text' and 'df' together. A left merge is used so that all rows for hospital admissions are included
# and any caregiver notes that are not associated with a hospital admission are dropped.
df2=pd.merge(grouped_text[['HADM_ID', 'TEXT']], df[['SUBJECT_ID', 'HADM_ID','LOS_HOSP', 'ADMITTIME', 'HOSPITAL_EXPIRE_FLAG',\
                                                    'ADMISSION_TYPE', 'DEATHTIME', 'ADMIT+24','ENDTIME']],\
             on=('HADM_ID') , how='left')
print('There are ', len(df2), 'rows and ',len(df2.columns), 'columns.')

There are  428039 rows and  10 columns.


In [41]:
df2 = df2.drop_duplicates()

In [42]:
print('There are now ', len(df2), 'rows and ', len(df2.columns), 'columns in df2.')

There are now  53977 rows and  10 columns in df2.


In [43]:
#check if any samples died before admission and remove them from the dataframe. Save as new dataframe.

df2.loc[df2['DEATHTIME'] < df2['ADMITTIME']]

,HADM_ID,TEXT,SUBJECT_ID,LOS_HOSP,ADMITTIME,HOSPITAL_EXPIRE_FLAG,ADMISSION_TYPE,DEATHTIME,ADMIT+24,ENDTIME
XXXXX,XXXXXX,[**XXXX-X-X**] X:XX PM\n CHEST (PORTABLE AP) ...,XXXX,2 days 22:51:00,XXXX-XX-XX XX:XX:00,1,URGENT,XXXX-XX-XX XX:XX:00,XXXX-XX-XX XX:XX:00,XXXX-XX-XX XX:XX:00


In [44]:
#remove individual that died before admission
df2 = df2.drop(df2[df2['HADM_ID'] == XXXXX].index)
df2.shape

(53976, 10)

The data is now ready to begin processing!

In [45]:
#Saving the merged dataframe.
df2.to_csv('df2.csv', index=False)

For use in savings calculations (later), calculate the average length of hospital stay.

In [46]:
LOS_hosp_mean = admissions.LOS_HOSP.mean()
print('The average hospitalization length for patients who stayed in the ICU was ', LOS_hosp_mean)

The average hospitalization length for patients who stayed in the ICU was  10 days 13:56:24.582697803


To also use in savings analysis, calculate the average number of days in the ICU per admission.

Read in ICU stays data so that we can determine average number of days in ICU

In [47]:
icu_stays = pd.read_csv('ICUSTAYS.csv.gz', compression='gzip' )
print(icu_stays.shape)
#icu_stays.head(2)

(61532, 12)


#### Columns of interest in the ICU dataset: 
- INTIME
- OUTTIME
- LOS (length of stay)
- HADM_ID

In [48]:
#convert OUTTIME, and INTIME to datetime format. The errors='coerce' argument allows for missing values
icu_stays.OUTTIME = pd.to_datetime(icu_stays.OUTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
icu_stays.INTIME = pd.to_datetime(icu_stays.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing outtime:', icu_stays.OUTTIME.isnull().sum())
print('Number of missing intime:', icu_stays.INTIME.isnull().sum())



Number of missing outtime: 10
Number of missing intime: 0


Remove the ICU stays with missing outtimes.

In [49]:
icu_stays = icu_stays.dropna(subset=['OUTTIME'])
print('Number of missing outtime:', icu_stays.OUTTIME.isnull().sum())


Number of missing outtime: 0


In [50]:
icu_stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61522 entries, 0 to 61531
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ROW_ID          61522 non-null  int64         
 1   SUBJECT_ID      61522 non-null  int64         
 2   HADM_ID         61522 non-null  int64         
 3   ICUSTAY_ID      61522 non-null  int64         
 4   DBSOURCE        61522 non-null  object        
 5   FIRST_CAREUNIT  61522 non-null  object        
 6   LAST_CAREUNIT   61522 non-null  object        
 7   FIRST_WARDID    61522 non-null  int64         
 8   LAST_WARDID     61522 non-null  int64         
 9   INTIME          61522 non-null  datetime64[ns]
 10  OUTTIME         61522 non-null  datetime64[ns]
 11  LOS             61522 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(6), object(3)
memory usage: 6.1+ MB


In [51]:
ICU_LOS_mean = round(icu_stays.LOS.mean(), 2)
print('average length of ICU stay: ', ICU_LOS_mean, 'days')

average length of ICU stay:  4.92 days


In [52]:
df_los = icu_stays[['HADM_ID', 'LOS']]
#df_los

In [53]:
print('There are ', len(df_los[df_los.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are  3746 duplicate hospital admissions.


These admissions are patients who stayed more than once in the ICU on that admission. LOS_ICU values for these patients need to be combined for a total LOS_ICU for each admission.

In [54]:
df_los.loc[df_los['LOS'] < 0]


,HADM_ID,LOS


In [55]:
#add column, LOS_ICU_TOTAL 
#df['LOS_ICU'] = df['LOS_ICU'].dt.days
#df.LOS_ICU

#df.LOS_ICU_TOTAL = np.where(df[df.duplicated(['HADM_ID'])], df['LOS_ICU'].sum(), df['LOS_ICU'])

#df['is_dup'] = df[['lat', 'lon']].duplicated()
#df['dups'] = df.groupby(['lat','lon']).is_dup.transform(np.sum)
# df outputs:

df_los = df_los.groupby(['HADM_ID'], as_index=False)['LOS'].sum()


In [56]:
df_los.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57776 entries, 0 to 57775
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   HADM_ID  57776 non-null  int64  
 1   LOS      57776 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.3 MB


In [57]:
print('There are now', len(df_los[df_los.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are now 0 duplicate hospital admissions.


In [58]:
ICU_LOS_mean_updated = round(df_los['LOS'].mean(), 2)

print('The average length of ICU stay was ', ICU_LOS_mean_updated, ' days.')

The average length of ICU stay was  5.24  days.
